## Kvantni klasifikator zasnovan na Groverovom algoritmu

Groverov algoritam se može koristi kao jednostavan klasifikator tako što će obeležiti nizove bitova koji predstavljaju pozitivnu klasu a zatim  pojačati njihove amplitude kako bi povećao verovatnoću njihovog merenja.

![GroverAlgorithm](images/GroverAlgorithm.png)

Algoritam počinje uniformnom superpozicijom (H). Orakl (O) invertuje tražena stanja, a difuzor (D) pojačava njihove amplitude. Završno merenje otkriva obeležena stanje sa visokom verovatnoćom.

---

### Zadatak

Koristeći podatke o pacijentima za testiranje gripa kodirane u binarnom obliku — temperatura, kašalj, antigen test — razviti Grover-ov kvantni klasifikator koji obeležava slučajeve pozitivne na grip. Pokrenuti algoritam i prikazati rezultate merenja kao raspodelu verovatnoća nad nizovima bitova koji predstavljaju pozitivne klase.

### Očekivani rezultat

- Prikaz kvantnog kola Grover-ovog klasifikatora.
- Histogram izlaznih rezultata.

### Eksperimentisanje

1. Proširiti `patient_data` dodavanjem dodatnog dijagnostičkog testa (npr. rezultat krvnog testa ili nivo zasićenosti kiseonikom), kodirati ga kao binarnu karakteristiku i posmatrati ponašanje Grover-ovog klasifikatora sa ulazima od 4 kubita.

2. Zameniti binarni `patient_data` realnim vektorima karakteristika i mapirati svaki uzorak u binarni string koristeći klinički smislen prag pre primene Grover-ovog klasifikatora.

   Primer:
        
```
feature_vectors = [ 
    {"temp": 39.1, "cough": 2.0, "antigen": 0.85}, 
    {"temp": 38.3, "cough": 7.0, "antigen": 0.12}, 
    {"temp": 36.8, "cough": 6.0, "antigen": 0.05}, 
]```

In [ ]:
from IPython.display import display

from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import circuit_drawer, plot_distribution
from math import floor, sqrt, pi

def oracle(marked):
    n = len(marked[0])
    qc = QuantumCircuit(n)
    for s in marked:
        for i, b in enumerate(s):
            if b == '0':
                qc.x(i)
        qc.h(n-1)
        qc.mcx(list(range(n-1)), n-1)
        qc.h(n-1)
        for i, b in enumerate(s):
            if b == '0':
                qc.x(i)
        qc.barrier() 
    return qc

def diffusion(n):
    qc = QuantumCircuit(n)
    qc.h(range(n))
    qc.x(range(n))
    qc.h(n-1)
    qc.mcx(list(range(n-1)), n-1)
    qc.h(n-1)
    qc.x(range(n))
    qc.h(range(n))
    qc.barrier() 
    return qc

def build_classifier(marked):
    n = len(marked[0]) # qubits
    m = len(marked)    # solutions
    k = floor((pi/4) * sqrt((2**n) / m)) # iterations
    grover = QuantumCircuit(n, n)
    grover.h(range(n))  # uniform superposition
    
    O = oracle(marked)
    D = diffusion(n)

    for _ in range(k):
        grover.compose(O, inplace=True)
        grover.compose(D,   inplace=True)

    grover.measure(range(n), range(n))
    return grover

# -----------------------------------------------
#                main program
# -----------------------------------------------

patient_data = {
    '000': 0,  # no fever, no cough, antigen test neg
    '001': 0,  # no fever, no cough, antigen test pos
    '010': 0,  # no fever, cough,    antigen test neg
    '011': 0,  # no fever, cough,    antigen test pos
    '100': 0,  # fever,   no cough,  antigen test neg
    '101': 1,  # fever,   no cough,  antigen test pos  → mark
    '110': 0,  # fever,   cough,     antigen test neg
    '111': 1   # fever,   cough,     antigen test pos  → mark
}

marked = [b for b, y in patient_data.items() if y == 1] 
grover = build_classifier(marked)
display(circuit_drawer(grover, output="mpl"))

# --- Simulate ---
simulator = AerSimulator()
results = simulator.run(grover).result()
counts = results.get_counts(grover)

display(plot_distribution(counts, title="Grover-based quantum classifier (flu example)"))
